## NCDC Spatial Distribution

In previous tutorial, students can access NCDC weather data by cdo_api_py module. We learned how to download precipitation data within a given extent boundary. 


In this tutorial, the statistics of NCDC daily precipitation data is presented by color mapping and changing marker size across the whole Indiana.  


In [ ]:
from cdo_api_py import Client
import pandas as pd
import datetime
# token for accessing the NCDC server
token = "<Your Token>"# be sure not to share your token publicly
# Client helps you access the NCDC database with your token
my_client = Client(token, default_units="None", default_limit=1000)

## Get raw dataframe of Stations in Indiana

Use the function learned in previous tutorial to download multiple stations in the give extent

In [ ]:
# write your code to get stations in the given extent in tutorial

## Clean Precipitation Dataset
There are hundreds NCDC weather stations in Indiana. However, many stations have low datacoverage. 

Our task in this part is to keep the dataset with sufficient observation (>95% datacoverage). Our list of stations contains stations in other state. Therefore, we have to get rid of those stations as well.

In [ ]:
dropind = []
# Drop station without enough date of observation
for i in range(len(stations.maxdate)):
    # get max and min date of each station
    date_str = stations.maxdate[i]
    date_str_min= stations.mindate[i]
    # transfer string to datetime
    datelast = datetime.datetime.strptime(date_str, '%Y-%m-%d')
    datefirst= datetime.datetime.strptime(date_str_min, '%Y-%m-%d')
    # get the position of stations with insufficient daily data
    if datelast-enddate < datetime.timedelta(days=0):
        dropind.append(i)
    elif datefirst-startdate > datetime.timedelta(days=0):
        dropind.append(i)        
# delete stations without enough time length
stations = stations.drop(stations.index[dropind])
stations_raw= stations

# Get names of indexes for which datacoverage less than 0.95
indexNames = stations[ stations['datacoverage'] < 0.95 ].index
# Delete these row indexes from dataFrame
stations.drop(indexNames , inplace=True)

# other gages with available data less than 0.95
# this list can be obtained after downloading all the datasets  
Insuff_gage = ['GHCND:US1INBN0010', 'GHCND:US1INBW0010',
                 'GHCND:US1INCW0003', 'GHCND:US1INLK0046',
                 'GHCND:US1INMN0007', 'GHCND:US1INMT0001',
                 'GHCND:US1INPS0001', 'GHCND:US1KYBT0001',
                 'GHCND:USC00116558', 'GHCND:USC00121873',
                 'GHCND:USC00122041', 'GHCND:USC00124244',
                 'GHCND:USC00126801']
# get rid of the stations in list above
# Get the final station list for downloading
stations_IN = stations[~stations['id'].isin(Insuff_gage)]
# get rid of the stations in other states
other_state = []
for i in stations_IN.index:
    if stations_IN['name'][i][-5:] != 'IN US':
        other_state.append(i)
stations_IN.drop(other_state , inplace=True)
# list first 20 stations to see if there is something wrong in the dataframe
stations_IN.head(20)

In [ ]:
# Get data from the NCDC client
i = 0
for rowid, station in stations_IN.iterrows(): 
    # try downloading due to some unaccessible sites of database
    try:
        station_data = my_client.get_data_by_station(
                        datasetid=datasetid,
                        stationid=station['id'],
                        startdate=startdate,
                        enddate=enddate,
                        return_dataframe=True,
                        include_station_meta=True)
        # set datetime to index
        station_data.set_index(pd.to_datetime(station_data['date']), inplace =True)
        # Drop all rows except for rainfall
        Rainfall_day = station_data.filter(['PRCP'])
        Rainfall_day = Rainfall_day.rename(columns={"PRCP": station['id']})
        # Merge the dataframe of rainfall from each station
        if i ==0:
            merged= Rainfall_day
        else:
            merged =pd.merge(merged,Rainfall_day ,how='outer', left_index=True, right_index=True)
        i +=1
    # print the id of broken dataset
    except:
        print(station['id'])
# The unit of rainfall is tenth of mm
# Get the final rainfall dataset
GHCN_IN  = merged

## Visualization of maximum precipitation

Use describe function to generate statistics of precipiation dataset. 

Drop the nan value of Describe_IN after append lat, long of stations to get a clean dataframe.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# There is a simple way to get statistics of dataframe 
Describe_IN = GHCN_IN.describe()
Describe_IN
# get lat, long values from stations dataframe
stations_lat = stations.pivot(columns='id', index = 'elevationUnit', values='latitude')
stations_long= stations.pivot(columns='id', index = 'elevationUnit', values='longitude')
# rename the index from meter to latitude and longitude
lat_IN  = stations_lat.rename(index={'METERS': 'latitude'})
long_IN = stations_long.rename(index={'METERS': 'longitude'})
# append lat, long dataframe to the statistics of all stations
Describe_IN = Describe_IN.append(lat_IN)
Describe_IN = Describe_IN.append(long_IN)
# show the statistics of stations in Indiana
Describe_IN = Describe_IN.T
Describe_IN = Describe_IN.dropna()

## Color Mapping
Color mapping maximum precipitation can have different color bar. In this script, 'jet' is the main cmap label and is the classic color bar from matlab.

In [ ]:
# set up the font size
plt.rcParams.update({'font.size': 15})
# create scatter plot of precipitation with color mapping of maximum values
fig, ax = plt.subplots()
Describe_IN.plot(kind="scatter", x="longitude", y="latitude", c='max', cmap="viridis",title='Max Precipitation in Indiana 2019', ax=ax)
plt.show()

In [ ]:
# write your code to get rid of the outlier and draw the scatter plot with color mapping of maximum precipitation

## Marker Size
Apply marker size to the maximum precipitation values. 

The formula of marker size can be changed to linear or exponential. It all depends on how you want to present your data.

In [ ]:
plt.rcParams.update({'font.size': 15})
Max_less = Describe_IN[Describe_IN['max']<2000]
# set up the size of maximum precipitation value
# we choose power of 5 to show a more obvious difference between data points
# remove the outlier
s = [5**(Max_less['max'][i]/Max_less['max'].mean()*2) for i in range(Max_less.shape[0])]
Max_less.plot(kind="scatter", x="longitude", y="latitude", s=s,title='Max Precipitation in Indiana 2019')
plt.show()

In [ ]:
# check the stations shows much larger maximum among others
Describe_IN[Describe_IN['max']>2000]